In [ ]:
%reload_kedro

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import array #useful to parse values
import pandas as pd

In [ ]:
IMGS_PATH = "/Users/gpalazzo/Desktop/dev/mba_Otavio/assets/images"

In [ ]:
model = catalog.load("logreg_fitted_model")

X_test = catalog.load("master_table_teste_ftes")
y_test = catalog.load("master_table_teste_tgt")
y_pred = catalog.load("logreg_model_predict")

df_model_rpt = catalog.load("logreg_model_relatorio")

In [ ]:
X_test.columns

In [ ]:
model.get_params()

In [ ]:
_roc_auc_score = roc_auc_score(y_true=y_test, y_score=y_pred)
_roc_auc_score

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])

plt.figure()

plt.plot(fpr, tpr, label="Regressão Logística (área = %0.2f)" % _roc_auc_score)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Taxa de Falso Positivo")
plt.ylabel("Taxa de Verdadeiro Positivo")

plt.title("Curva ROC: Regressão Logística")
plt.legend(loc="lower right")

plt.savefig(f"{IMGS_PATH}/logreg_curva_roc.png")

In [ ]:
cm = eval(df_model_rpt["confusion_matrix"][0])

ax = plt.subplot()
plot = sns.heatmap(cm, annot=True, ax=ax)
ax.set_xlabel("Label de Predição")
ax.set_ylabel("Label Real")
ax.set_title("Matriz de Confusão: Regressão Logística")

fig = plot.get_figure()
fig.savefig(f"{IMGS_PATH}/logreg_cm.png")

In [ ]:
cm

In [ ]:
probas = eval(df_model_rpt["test_probas"].values[0])

probas_df = pd.DataFrame.from_dict(data=probas, orient="index")
probas_df = probas_df.sort_index()

df = probas_df.merge(y_test, left_index=True, right_index=True, how="inner")
assert df.shape[0] == probas_df.shape[0] == y_test.shape[0]
df2 = df.merge(y_pred, left_index=True, right_index=True, how="inner")
assert df.shape[0] == df2.shape[0]

df_right = df2[df2["label"] == df2["y_pred"]]
df_wrong = df2.drop(df_right.index)

df_right = df_right.drop(columns=["label"])
right_probas = df_right[["proba_label_0", "proba_label_1"]].max(axis=1)

df_wrong = df_wrong.drop(columns=["label"])
wrong_probas = df_wrong[["proba_label_0", "proba_label_1"]].max(axis=1)

In [ ]:
plot = right_probas.hist()

fig = plot.get_figure()
fig.savefig(f"{IMGS_PATH}/hist_proba_label_correta.png")

In [ ]:
plot = wrong_probas.hist()

fig = plot.get_figure()
fig.savefig(f"{IMGS_PATH}/hist_proba_label_errada.png")

In [ ]:
fte_imp = eval(df_model_rpt["fte_importance"][0])["importance"]

data = {"features": fte_imp.keys(),
       "importance": fte_imp.values()}

df_fte_imp = pd.DataFrame.from_dict(data=data)
df_fte_imp = df_fte_imp.sort_values(by="importance", ascending=False)

print(df_fte_imp)